# Precificação 
Com o intuito de precificar imóveis, vamos buscar por modelos que expliquem bem o preço com base nas variáveis selecionadas. Com base em  pesquisas de casos semelhantes modelos como RandomForest e XGBoost parecem trazer bons resultados para esse tipo de problema. Mas também testarei o modelo knn que é mais simples e pode servir como um baseline de comparação com modelos mais complexos.

In [1]:
## Importing packages
import pandas as pd
import numpy as np

import utils as ut

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import warnings
warnings.filterwarnings("ignore")

In [2]:
x_train = pd.read_csv('data/x_train.csv').drop(columns=['Unnamed: 0'])
x_test = pd.read_csv('data/x_test.csv').drop(columns=['Unnamed: 0'])
y_train = pd.read_csv('data/y_train.csv')[['price']]
y_test = pd.read_csv('data/y_test.csv')[['price']]

In [8]:
model_registry = pd.DataFrame(columns= ['model', 'params', 'mse','rmse','mae','r2'])

## Pré processamento

criando um pre processador, para fazer a padronização das variáveis numéricas e o one hot encoding das variáveis categóricas.

Todos os modelos terão este pré processamento

In [4]:
colunas_numericas = x_train.select_dtypes(include=[np.number]).columns
colunas_categoricas = x_test.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), colunas_numericas),
        ('cat', OneHotEncoder(), colunas_categoricas)
    ],
    remainder='passthrough'  # Mantém as outras colunas não transformadas
)

## KNN

O algoritmo knn, funciona com base em calculo de distãncias. Ele obtem a distãncia entre um ponto e todos os outros e através dos k pontos mais próximos, ele infere o novo ponto como o valor médio entre os k vizinhos.

In [6]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', KNeighborsRegressor())
])

parametros = {'regressor__n_neighbors': np.arange(1, 11)}

# Criar um objeto GridSearchCV para encontrar o melhor valor de k
grid_search_knn = GridSearchCV(pipeline, parametros, cv=5, scoring='r2', refit=True)
grid_search_knn.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('num',
                                                                         StandardScaler(),
                                                                         Index(['host_is_superhost', 'instant_bookable', 'accommodates', 'bathrooms',
       'bedrooms', 'beds', 'review_scores_rating'],
      dtype='object')),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         Index(['cancellation_policy', 'property_type', 'room_type', 'price_region'], dtype='object'))])),
                                       ('regressor', KNeighborsRegressor())]),
             param_grid={'regressor__n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])},
             scoring='r2')

In [9]:
model_registry = ut.registro_metricas(model_name= 'Knn',
                  df_init = model_registry,
                  y_test=  y_test,
                  y_pred = grid_search_knn.predict(x_test),
                  best_param= str(grid_search_knn.best_params_))
model_registry


,model,params,mse,rmse,mae,r2
0,Knn,{'regressor__n_neighbors': 9},25185.690195,158.700001,51.206364,0.47302


O Algoritmo mais simples consegue explicar cerca de 47% da variabilidade do preço em relação as variáveis utilizadas.

## Random Forest

Random forest é um técnica de ensamble bastante robusta, onde ele combina a previsão de várias árvores de decisão e define o preço a partir do resultado predito por várias árvores.

In [10]:
## Random forest

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=2))
])

parametros = {
    'regressor__n_estimators': [40,50, 100, 200],
    'regressor__max_depth': [None,5, 10],
    'regressor__min_samples_split': [15,20,25],
    'regressor__max_features': ['auto', 'sqrt', 'log2']
}


grid_search_rf = GridSearchCV(pipeline, parametros, cv=5, scoring='r2', refit=True)
grid_search_rf.fit(x_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('num',
                                                                         StandardScaler(),
                                                                         Index(['host_is_superhost', 'instant_bookable', 'accommodates', 'bathrooms',
       'bedrooms', 'beds', 'review_scores_rating'],
      dtype='object')),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         Index(['cancellation_policy', 'property_type', 'room_type', 'price_region'], dtype='object'))])),
                                       ('regressor',
                                        RandomForestRegressor(random_state=2))]),
             param_grid={'regressor__max_depth': [None, 5, 10],
                         'regressor__max_features': ['auto', 'sqrt', 'log2'],
                         'regressor__min_samples_split': [15, 20, 25],
                         'regressor__n_estimators': [40, 50, 100, 200]},
             scoring='r2')

In [11]:
model_registry = ut.registro_metricas(model_name= 'Random Forest',
                  df_init = model_registry,
                  y_test=  y_test,
                  y_pred = grid_search_rf.predict(x_test),
                  best_param= str(grid_search_rf.best_params_))
model_registry

,model,params,mse,rmse,mae,r2
0,Knn,{'regressor__n_neighbors': 9},25185.690195,158.700001,51.206364,0.473020
1,Random Forest,"{'regressor__max_depth': None, 'regressor__max...",16162.831034,127.133123,34.317675,0.661812


## XGBoost

o Extreme gradient boosting funciona a partir do treinamento sequencial de diversas árvores de decisões, e a cada novo modelo, tenta corrigir os erros dos modelos anteriores.

In [12]:
# Criar o pipeline com o transformador e o regressor XGBRegressor
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(random_state=2))
])

parametros = {
    'regressor__n_estimators': [50, 100, 200,300],
    'regressor__max_depth': [3, 5, 10],
    'regressor__learning_rate': [0.01, 0.1, 0.2],
    'regressor__subsample': [0.7,0.8, 0.9],
    'regressor__colsample_bytree': [0.7,0.8, 1.0]
}

# Criar um objeto GridSearchCV para encontrar os melhores parâmetros
grid_search_xg = GridSearchCV(pipeline, parametros, cv=5, scoring='r2', refit=True)
grid_search_xg.fit(x_train, y_train)



GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('num',
                                                                         StandardScaler(),
                                                                         Index(['host_is_superhost', 'instant_bookable', 'accommodates', 'bathrooms',
       'bedrooms', 'beds', 'review_scores_rating'],
      dtype='object')),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         Index(['cancellation_policy', 'property_type', 'room_type...
                                                     missing=nan,
                                                     monotone_constraints=None,
                                                     multi_strategy=None,
                                                     n_estimators=None,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     random_state=2, ...))]),
             param_grid={'regressor__colsample_bytree': [0.7, 0.8, 1.0],
                         'regressor__learning_rate': [0.01, 0.1, 0.2],
                         'regressor__max_depth': [3, 5, 10],
                         'regressor__n_estimators': [50, 100, 200, 300],
                         'regressor__subsample': [0.7, 0.8, 0.9]},
             scoring='r2')

In [13]:
model_registry = ut.registro_metricas(model_name= 'XGBoost',
                  df_init= model_registry,
                  y_test=  y_test,
                  y_pred = grid_search_xg.predict(x_test),
                  best_param= str(grid_search_xg.best_params_))
model_registry

,model,params,mse,rmse,mae,r2
0,Knn,{'regressor__n_neighbors': 9},25185.690195,158.700001,51.206364,0.473020
1,Random Forest,"{'regressor__max_depth': None, 'regressor__max...",16162.831034,127.133123,34.317675,0.661812
2,XGBoost,"{'regressor__colsample_bytree': 0.7, 'regresso...",20838.923980,144.356933,38.306938,0.563971


## Resultados

Dentro os modelos testados, a Random Forest foi o que apreentou melhor desempenho. 
No momento o melhor modelo explica 66% da variabilidade do modelo, no entanto, ainda há mais abordagens que podem ajudar a melhorar o desempenho, por exemplo, adicionar fatores geograficos na análise, seja via agrupamento de bairros por proximidade, ou por regiões mais turisticas. Ainda seria interessante avaliar as variáveis cuja interpretabilidade não foi possível.

Outro fator que poderia ser interessante para a análise, seria a adição de informações do valor de acordo com a época do ano.

## Sugestão de produtização

A produtização do modelo poderia ser feita via API, uma ótima opção neste caso, seria ter feito o treinamento do modelo utilizando o MLFLOw, que ja disponibiliza o modelo via API, além do ML FLow, existem outras formas de disponibilizar o modelo, como por exemplo, criando uma API com o Fast API e utilizando azure functions para aceitar as requisições.

A utilização do Kedro também seria interessante no quesito de organização de código e treinamento do modelo, inclusive, podendo usá-lo em conjunto com o MLFlow.

Quanto a interface, pensando num cenário de grande volume de dados seria interessante ter alguma aplicação web que faz requisições na API, em um cenário mais simples, o próprio python dispões de pacotes que possibilitam a criação da interface, como por exemplo o streamlit, onde seria possível disponibilizar de forma gratuita e sem necessidade de uma API Externa.

Tanto a criação da API, quanto a utilização de algum pacote de visualização como o streamlit, trariam maior estabilida e segurança no desenvolvimento caso optasse pelo desenvolvimento dentro de um dev container, ou com a criação de um container docker para armazenar a aplicação.

Por fim,  independente de onde e como for feita a produtização, é importante criar um mecanismo de monitoramento do modelo, para validar se ele permanece aderente aos dados ou se precisa ser retreinado. O monitoramento pode ser feito a partir de testes de hipótese para validar se a distribuição dos dados permanece a mesma.

## Outras ideias

Pensando no contexto de acomodações do airbnb,um ponto interesane a ser abordado caso haja a possibilidade de buscar dados dos usuários, seria a criação de um sistema de recomendações de acomodações, podendo ser feito tanto com base em outras acomodações ja locadas pelo usuário, quanto em sugestões baseadas na similaridade entre usuários.